In [1]:
from langchain.chat_models import  ChatOpenAI
from langchain.agents import initialize_agent,AgentType, Tool
from langchain.utilities import SQLDatabase
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List, Optional, Any
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 获取 环境变量
#大模型apikey和url
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

#天气查询的apikey
juheAppKey = os.getenv('juheAppKey')

#数据库信息
mysql_info = os.getenv('mysql_info')

In [3]:
#定义大模型
online_model = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_9764/113483475.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  online_model = ChatOpenAI(


In [4]:
#定义数据库 连接
db = SQLDatabase.from_uri(mysql_info)


In [6]:
#查看数据库中的可用表
db.get_usable_table_names()

['city_stats']

In [9]:
#查看表结构
print(db.get_table_info())


CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/


In [12]:
#查看数据库信息
print(db.get_context())

{'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}


In [13]:
#定义数据库信息查询工具
def get_database_info(_):
    return db.get_context()

#定义数据库查询工具
def get_database_query(query):
    return db.run(query)

In [15]:
#获取 sql 语句 链
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough,RunnableParallel

template_sql="请通过写sql代码来回答对应问题，并且需要基于如下数据库信息：{info} \n 需要回答问题是：{question} \n 注意仅需要通过sql代码回答，不需要文字 \n 代码形式如下: ```sql\n...\n```"
t_sql=PromptTemplate.from_template(template_sql)

#获取 大模型生成的 sql 语句，并进行提取
def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

chain_sql=({"info":get_database_info,"question":RunnablePassthrough()}| t_sql | online_model | StrOutputParser() | RunnableLambda(get_sql))

In [17]:
#Test
result = chain_sql.invoke({"question":"请帮我查询一下城市表中的所有信息"})
result

'\nSELECT * FROM city_stats;\n'

In [18]:
#获取 sql 语句执行结果 链
template_sql0="请通过综合如下的数据库信息，问题，sql代码，sql代码的执行结果给出问题的自然语言回答：\n 数据库信息{info} \n 需要回答问题是：{question} \n sql代码：{query} \n sql代码执行结果: {res}"
t_sql0=PromptTemplate.from_template(template_sql0)

#完整流程,获取数据库信息，生成sql语句，执行sql语句，获取结果，生成回答
chain_sql0=( {"info":get_database_info,"question":RunnablePassthrough(),"query":chain_sql}
            |RunnablePassthrough.assign(res=lambda x: get_database_query(x["query"])) 
            | t_sql0 | online_model |StrOutputParser())

#执行完整流程
response = chain_sql0.invoke("请告诉我数据库中人口最多的两个城市是哪些，分别是多少人？")
print(response)

根据提供的SQL查询结果，数据库中人口最多的两个城市及其人口分别是：

1. **北京**，人口为 **20,000,000** 人。  
2. **河北**，人口为 **19,999,944** 人。

因此，人口最多的两个城市是 **北京** 和 **河北**，对应的人口数量分别为 **2000万** 和 **近2000万**。
